In [ ]:
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
from scipy.sparse import spdiags,linalg,eye
J=1      
h=0

# define the initia configuration
def initialstate(L):
    state = 2*np.random.randint(2, size=(L,L))-1
    return state
def update(config,beta):
    for i in range(L):
        for j in range(L):
            # choose a random cell in the grid
            cell = np.random.randint(L, size=2)
            S=config[cell[0],cell[1]]
            # calculate the current contribution of this cell to the total energy
            energy_cell = 0
            # be careful with the boundary
            if cell[0] != 0:   energy_cell += config[cell[0]-1, cell[1]]   # left neighbor
            else:   energy_cell += config[L-1, cell[1]]
            if  cell[0]!= L-1: energy_cell += config[cell[0]+1, cell[1]]# right neighbor
            else:   energy_cell += config[0, cell[1]]
            if cell[1] != 0:   energy_cell += config[cell[0], cell[1]-1]   # top neighbor
            else:   energy_cell += config[cell[0], L-1]
            if cell[1] != L-1: energy_cell += config[cell[0], cell[1]+1]   # bottom neighbor
            else:   energy_cell += config[cell[0], 0]

            # the current contribution to the interaction energy is: -J*energy_cell*config[i,j]
            # if we flip the spin, i.e. change config[i,j] to -config[i,j], the new contribution will also change sign
            # this means that overall:

            energy_difference = 2*J*energy_cell*S

            # additional contribution due to the magnetic field
            energy_difference += 2*h*S

            # check if update is accepted
            accept = False
            if energy_difference<0: 
                accept = True    # always accept updates that decrease the energy
            else:
                prob = np.exp(-beta*energy_difference)
                rand = np.random.rand()
                if rand<prob:
                    accept = True

            # if update is accepted we flip the spin
            if accept == True:
                config[cell[0],cell[1]] = -S

            # keep track of the acceptance probability
    return config
def energy():  
    # elegant way of writing the sum over all nearest neighbors in the grid
    interaction = -J*( np.sum(config[0:L-1,:]*config[1:L,:]) +     # contributions of top/bottom neighbors
                       np.sum(config[:,0:L-1]*config[:,1:L]))     # contributions of left/right neighbors
    
    magnetic = -h*np.sum(config)
    return (interaction + magnetic)/2
def magnetization():
    return np.sum(config)


nt      = 50         #  number of temperature points
L       = 10       #  size of the lattice, L x L
eqSteps = 4000 #  number of MC sweeps for equilibration
mcSteps = 4000    #  number of MC sweeps for calculation


T       = np.linspace(0.5, 6.0, nt); 
E,M,C,X = np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt)
error_M=np.zeros(nt)
n1, n2  = 1.0/(mcSteps*L*L), 1.0/(mcSteps*mcSteps*L*L) 
# divide by number of samples, and by system size to get intensive values
#  MAIN PART OF THE CODE
#----------------------------------------------------------------------
magnetization_results =np.zeros(mcSteps)
f = plt.figure(figsize=(18, 15))

config_data=[]

for tt in range(nt):
    config = initialstate(L) # initialise
    
    TEM=T[tt]

    E1 = M1 = E2 = M2 = 0
    #storing results for error calculation
    magnetization_results=[] 
    
    
    iT=1.0/T[tt]; iT2=iT*iT;
    
    for i in range(eqSteps):         # equilibrate
        update(config,iT)           # Monte Carlo moves

    for i in range(mcSteps):
        update(config, iT)           
        Ene = energy()     # calculate the energy
        Mag = magnetization()  # calculate the magnetisation
        magnetization_results.append(Mag/(L*L))
        
        if i==10:                          #storing configuration for congiguration plot

            config_data.append(np.copy(config))
        

        E1 = E1 + Ene
        M1 = M1 + Mag
        M2 = M2 + Mag*Mag 
        E2 = E2 + Ene*Ene
    sp =  f.add_subplot(3, 2, 5 )
    if TEM== T[0]:
        plt.plot(magnetization_results[3000:], color='RoyalBlue',label='T={}'.format(T[0]))
        plt.xlabel("Monte Carlo time", fontsize=20); 
        plt.ylabel("Magnetization per spin", fontsize=20);   plt.axis('tight');
        plt.legend()
        
    if TEM==T[-1]  :
        plt.plot(magnetization_results[3000:], color='IndianRed',label='T={}'.format(T[-1]))
        plt.xlabel("Monte Carlo time", fontsize=20); 
        plt.ylabel("Magnetization per spin", fontsize=20);   plt.axis('tight');
        plt.legend()
        plt.savefig('Magnetisation')

    # divide by number of sites and iteractions to obtain intensive values    
    E[tt] = n1*E1
    M[tt] = n1*M1
    C[tt] = (n1*E2 - n2*E1*E1)*iT2
    X[tt] = (n1*M2 - n2*M1*M1)*iT
    
    # error calculation
    SqrSum_M=0
    
    for i in range(len(magnetization_results)):
        SqrSum_M+=(magnetization_results[i]-M[tt])**2
        
    var_M=SqrSum_M/(mcSteps-1)
    
    error_M[tt]=np.sqrt(var_M/mcSteps)
    
    
        
        
    

#  plot the calculated values  

conditon = (C== np.max(C))
maxpos = np.where(conditon)

sp =  f.add_subplot(3, 2, 1 );
plt.scatter(T, E, s=50, marker='o', color='IndianRed')


plt.xlabel("$K_B$T/J", fontsize=20);
plt.ylabel("Energy per spin/J ", fontsize=20); 
plt.axis('tight')
plt.savefig('energy')


sp =  f.add_subplot(3, 2, 2 );
#plt.scatter(T, abs(M), s=50, marker='o', color='RoyalBlue')
plt.errorbar(T, abs(M), yerr = error_M,fmt='o',ecolor = 'RoyalBlue',color='black')
plt.vlines(T[maxpos[0]],ymin=0.0, ymax=.8, linestyle='--',color='r',transform = sp.get_xaxis_transform())
plt.text(T[maxpos[0]], 0.0, "T= %.3f" % (T[maxpos[0]]),  verticalalignment='center')
plt.xlabel("$K_B$T/J", fontsize=20); 
plt.ylabel("Magnetization ", fontsize=20);   plt.axis('tight');
plt.savefig('magnetisation_pr')


sp =  f.add_subplot(3, 2, 3 );
plt.scatter(T, C, s=50, marker='o', color='IndianRed')
plt.vlines(T[maxpos[0]],ymin=0.0, ymax=0.1, linestyle='--',color='r',transform = sp.get_xaxis_transform())
plt.text(T[maxpos[0]], 0.0, "T= %.3f" % (T[maxpos[0]]),  verticalalignment='center')

plt.xlabel("$K_B$T/J", fontsize=20);  
plt.ylabel("Specific Heat per spin/$K_B$ ", fontsize=20);   plt.axis('tight');
plt.savefig('specific heat')


sp =  f.add_subplot(3, 2, 4 );
plt.scatter(T, X, s=50, marker='o', color='RoyalBlue')
plt.vlines(T[maxpos[0]], ymin=0.0, ymax=25.0,linestyle='--',color='r',transform = sp.get_xaxis_transform())
plt.text(T[maxpos[0]], 0.0, "T= %.3f" % (T[maxpos[0]]),  verticalalignment='center')
plt.xlabel("$K_B$T/J", fontsize=20); 
plt.ylabel("Susceptibility", fontsize=20);   plt.axis('tight');
plt.savefig('Susceptibility')

fig, ax = plt.subplots(2, 2, figsize=(8, 8))
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
ax = ax.ravel()
for i in range(0,nt,13):
    ax[(i%12)].imshow(config_data[i])
    ax[i%12].set_xlim(0,L)
    ax[i%12].set_ylim(0,L)
    ax[(i%12)].set_title('T={}'.format(round(T[i],2)))
    ax[i%12].set_aspect('equal', 'datalim')
    fig.savefig('diff_tem')